In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import  Distance, VectorParams, PointStruct

import pandas as pd
import openai

/home/arminas_work/projects/AI-Engineering_Bootcamp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read the sampled dataset with Amazon inventory data

In [2]:
df_items= pd.read_json('../../data/meta_Electronics_2022_2023_with_category_ratings_100_sample_1000.jsonl', lines=True)

In [3]:
len(df_items)

1000

In [ ]:
df_items.head()

In [ ]:
list(df_items["features"].items())[:2]

In [ ]:
list(df_items["images"].items())[0]

### Preprocess title and features

In [ ]:
def make_description(row):
    return f"{row['title']}{" ". join(row['features'])}"

In [ ]:
def extract_large_image(row):
    return row['images'][0].get('large', '')

In [ ]:
df_items['description'] = df_items.apply(make_description, axis=1)
df_items['image'] = df_items.apply(extract_large_image, axis=1)

In [ ]:
df_items.head()


In [ ]:
list(df_items["description"].items())[:2]

### Get 50 items Sample

In [ ]:
df_sample = df_items.sample(50, random_state=42)

In [ ]:
len(df_sample)

In [ ]:
data_to_embed= df_sample[['description', 'image', 'rating_number', 'price', 'average_rating', 'parent_asin']]

In [ ]:
data_to_embed.head()

In [ ]:
data_to_embed = data_to_embed.to_dict(orient='records')

In [ ]:
data_to_embed

### Embedding function

In [ ]:
def get_embedding(text:str):
    response =openai.embeddings.create(
        input=text,
        model='text-embedding-3-small'
    )
    return response.data[0].embedding

In [ ]:
get_embedding('Hi')

### Create Qdrant collection

In [ ]:
qdrant_client = QdrantClient(url="http://localhost:6333")

In [ ]:
qdrant_client.create_collection(
    collection_name="Amazon-items-collection-00",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
)

### Embed data

Test

In [ ]:
pointstruct = PointStruct(
    id=0,
    vector= get_embedding("Test text"),
    payload = {
        "text": "Test text",
        "model": "text-embedding-3-small",
    }
)

In [ ]:
pointstruct.payload

### Amazon data

In [ ]:
pointstructs = []
for i, data in enumerate(data_to_embed):
    embedding = get_embedding(data['description'])
    pointstructs.append(
        PointStruct(
            id=i,
            vector=embedding,
            payload=data,
        )
    )

In [ ]:
len(pointstructs)

### Write embedded data to Qdrant

In [ ]:
qdrant_client.upsert(
    collection_name="Amazon-items-collection-00",
    wait=True,
    points=pointstructs
)

### Retreive data from Qdrant

In [ ]:
def retreive_data(query: str, k=5):
    query_embedding = get_embedding(query)
    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k
    )

    return results

Test

In [ ]:
retreive_data("what kind of charging cords do you offer").points